
General intuition of the code:
- we are simulating a world with agents on a network who choose between EV and gas cars; their decisions depend on (1) what their neighbours do and (2) how much charging infrastructure exists

Plotting functions

# Ratio sweep

One ratio & outcome; reveals tipping points; You vary the initial ratio of EV adopters (or infrastructure level) while keeping other parameters fixed.

You’ve found bistability if:

For some mid-range values, some runs end low and some end high

The results are not a smooth curve but split into two clusters

📌 Two separate plateaus = bistability.
- phase plot: X0 & ration; reveals tipping points and bistability
- spaghetti plot: many simulations; shows how different runs behave over time; reveals multiple stable states (some runs go to 0 some to 1)
- density plot:  similar to spaghetti but smoothed; shows where trajectories cluser

You sweep X₀, I₀, and βᵢ. You uncover:
- critical tipping points
- the size of the bistable region
- how hard (or easy) diffusion is
- how infrastructure feedback matters

From this you conclude: “The system naturally gets stuck unless certain conditions are met.”
This motivates why policy is needed.

Then you rerun the sweeps for different network types. You learn:
- Some networks are more resilient
- Some spread adoption better
- BA networks may need lower X₀
- Random networks may need higher initial adoption

This tells you: “Policy needs to be targeted differently depending on the social structure.”
For example: “Target high-degree hubs first in a BA network.”

Then for the policy design we use the insights from the first 2 parts.

In [1]:
import numpy as np
import pandas as pd

from ev_experiments import (
    run_timeseries_trial,
    ratio_sweep_df,
    phase_sweep_df,
    traces_to_long_df,
    collect_intervention_trials
)

from ev_plotting import (
    plot_ratio_sweep,
    plot_phase_plot,
    plot_spaghetti,
    plot_density
)


# "Sanity" check of the code

In [3]:
X, I, df = run_timeseries_trial( # calls the function that constructs the stag hunt model 
    T=200,
    scenario_kwargs={ # a dictionary that overrides the model's default paramteres 
        "ratio": 2.0,    # initial payoff ratio (aI/b)
        "X0_frac": 0.2,  # 20% initial EV adopters
        "network_type": "random",
        "n_nodes": 200,
    }
)

df.head()


,X,I
0,0.0,0.045000
1,0.0,0.040500
2,0.0,0.036450
3,0.0,0.032805
4,0.0,0.029525


# Baseline scenario dictionary

In [2]:
scenario_base = dict( # creates a dictionary of parameters we can pass to sweep fuctions
    I0=0.05, # infrastructure level 
    beta_I=2.0, # how strongly infrastructure increases the coordination payoff a_i = a0 + beta_I * I
    b=1.0, # baselnie payoff for D (ICE); used to compute a_i/b
    g_I=0.05, # infrastructure adjustment rate (how quick infrastructure responds to adoption)
    network_type="BA",    # You can change this later for Part 2
    n_nodes=120,
    p=0.05, # edge probability (ER)
    m=2, # number of edges to attach for new nodes (BA)
)


# Parameter sweep

In [ ]:
X0_values = np.linspace(0, 1, 21) # Creates 21 evenly spaced initial adoption fractions from 0.0 to 1.0
ratio_values = np.linspace(0.8, 3.5, 41) # Creates 41 values for the payoff ratio a_I / b to sweep across plausible values.

phase_df = phase_sweep_df( # runs for each ratio and each X0 value and returns a df with X0, ration, X_final as columns
    X0_values=X0_values,
    ratio_values=ratio_values,
    scenario_kwargs=scenario_base, # uses baseline parameters
    batch_size=3,       # lower = faster
    T=200,
    strategy_choice_func="logit", # uses probabilistic update wich affects stochasticity 
    tau=1.0, # temperature for the logit rule (higher = more randomness)
)

plot_phase_plot(phase_df) # key visual for tipping points and bistability


# Ratio sweep (final X* vs ration for fixed X0)

In [8]:
sweep_df = ratio_sweep_df( # for a fixed final adoption 0.2 this runs trials for every ratio and computes the mean final adopton X*
    X0_frac=0.20,
    ratio_values=np.linspace(0.8, 3.5, 41),
    scenario_kwargs=scenario_base,
    T=200,
)

plot_ratio_sweep(sweep_df) # A sharp vertical step (or steep slope) in this curve indicates a tipping point for that fixed X0_frac — a small change in ratio causes a big change in final adoption.


'c:\\Users\\ecate\\Desktop\\Assignment 3\\Assignment-3-MBDM\\plots\\ev_ratio_sweep.png'

# Network structure comparison

In [7]:
for net in ["random", "BA"]: # loop over 2 network types
    print("Network:", net)

    phase_df = phase_sweep_df(
        X0_values=np.linspace(0, 1, 21),
        ratio_values=np.linspace(0.8, 3.5, 41),
        scenario_kwargs={**scenario_base, "network_type": net},
        batch_size=3,
        T=200
    )

plot_phase_plot(phase_df)


Network: random
Network: BA


'c:\\Users\\ecate\\Desktop\\Assignment 3\\Assignment-3-MBDM\\plots\\ev_phase_plot.png'

# Infrastructure sweep

In [ ]:
I0_list = [0.0, 0.05, 0.1, 0.2, 0.3]

all_phase_plots = []

for I0 in I0_list:
    print("Running for I0 =", I0)
    
    scenario_I0 = {**scenario_base, "I0": I0}
    
    phase_df = phase_sweep_df(
        X0_values=np.linspace(0,1,21),
        ratio_values=np.linspace(0.8,3.5,41),
        scenario_kwargs=scenario_I0
    )
    
plot_phase_plot(phase_df)


Running for I0 = 0.0
Running for I0 = 0.05
Running for I0 = 0.1


# Policy intervention example

subsidy = dict(start=20, end=80, delta_a0=0.4)

baseline_X, baseline_I, subsidy_X, subsidy_I, base_df, subs_df = \
    collect_intervention_trials(
        n_trials=50,
        T=200,
        scenario_kwargs=scenario_base,
        subsidy_params=subsidy,
    )

traces_df = traces_to_long_df(baseline_X, subsidy_X)
plot_spaghetti(traces_df)
plot_density(traces_df)
